# Multi-Agent Support
This is an example implementation of tracking events from two separate agents

In [1]:
import agentops
from agentops.agent import track_agent
from dotenv import load_dotenv
import os
import openai
import logging
import traceback

from IPython.display import display, Markdown

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', "<your_openai_key>")
AGENTOPS_API_KEY = os.getenv('AGENTOPS_API_KEY', "<your_agentops_key>")
logging.basicConfig(level=logging.INFO)

In [3]:
ao_client = agentops.Client(AGENTOPS_API_KEY, endpoint='http://localhost:8000')
openai_client = openai.Client()

INFO:root:View info on this session at https://agentops.ai/dashboard?session_id=47aef9e1-94d3-4e69-8d75-d40c453b5b74


Now lets create a few agents!

In [ ]:
res = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": "You are not a tracked agent"},
    {"role": "user", "content": "Say hello"}]
)
# traceback.print_exc()
# raise Exception()
res.choices[0].message.content

In [6]:
@track_agent(name='qa')
class QaAgent:
    def completion(self, prompt: str):
        res = openai_client.chat.completions.create(model='gpt-3.5-turbo', messages=[{"role": "system", "content": "You are a qa engineer and only output python code, no markdown tags."},
    {"role": "user", "content": prompt}], temperature=0.5)
        return res.choices[0].message.content
        
@track_agent(name='engineer')
class EngineerAgent:
    def completion(self, prompt: str):
        res = openai_client.chat.completions.create(model='gpt-3.5-turbo', messages=[{"role": "system", "content": "You are a software engineer and only output python code, no markdown tags."},
    {"role": "user", "content": prompt}], temperature=0.5)
        return res.choices[0].message.content

In [7]:
qa = QaAgent()
engineer = EngineerAgent()

In [8]:
engineer._ao_agent_name

'engineer'

In [9]:
generated_func = engineer.completion("Write a python function that accepts two numbers and multiplies them together, then divides by two. No example.")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:LLM call from agent named: engineer


ChatCompletion(id='chatcmpl-90J3B0QxoJxMWjCGnqOKjXLAWwk3T', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='def multiply_and_divide(num1, num2):\n    result = (num1 * num2) / 2\n    return result', role='assistant', function_call=None, tool_calls=None))], created=1709860257, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_2b778c6b35', usage=CompletionUsage(completion_tokens=28, prompt_tokens=48, total_tokens=76))
{'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': 'You are a software engineer and only output python code, no markdown tags.'}, {'role': 'user', 'content': 'Write a python function that accepts two numbers and multiplies them together, then divides by two. No example.'}], 'temperature': 0.5}
2024-03-08T01:10:56.869Z
<class 'openai.Stream'>
<class 'openai.AsyncStream'>
<class 'openai.types.chat.chat_completion_chunk.ChatCompletionChunk'>
<class 'openai.resources.completions.As

In [10]:
display(Markdown('```python\n' + generated_func + '\n```'))


```python
def multiply_and_divide(num1, num2):
    result = (num1 * num2) / 2
    return result
```

In [11]:
generated_test = qa.completion("Write a python unit test that test the following function: \n " + generated_func)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:LLM call from agent named: qa


ChatCompletion(id='chatcmpl-90J3COBfcX26PVyuihVOXdOvG3jJW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="import unittest\n\ndef multiply_and_divide(num1, num2):\n    result = (num1 * num2) / 2\n    return result\n\nclass TestMultiplyAndDivide(unittest.TestCase):\n\n    def test_multiply_and_divide(self):\n        self.assertEqual(multiply_and_divide(4, 2), 4)\n        self.assertEqual(multiply_and_divide(0, 5), 0)\n        self.assertEqual(multiply_and_divide(-3, 6), -9)\n\nif __name__ == '__main__':\n    unittest.main()", role='assistant', function_call=None, tool_calls=None))], created=1709860258, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_2b778c6b35', usage=CompletionUsage(completion_tokens=113, prompt_tokens=66, total_tokens=179))
{'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': 'You are a qa engineer and only output python code, no markdown tags.'}, {'role': 'user', 'cont

In [12]:
display(Markdown('```python\n' + generated_test + '\n```'))

```python
import unittest

def multiply_and_divide(num1, num2):
    result = (num1 * num2) / 2
    return result

class TestMultiplyAndDivide(unittest.TestCase):

    def test_multiply_and_divide(self):
        self.assertEqual(multiply_and_divide(4, 2), 4)
        self.assertEqual(multiply_and_divide(0, 5), 0)
        self.assertEqual(multiply_and_divide(-3, 6), -9)

if __name__ == '__main__':
    unittest.main()
```